In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import tensorflow as tf
from models import CNN_Model

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
Using TensorFlow backend.


In [2]:
# Load trained model
weights_path = '../models/trained_mvp_model_benchmark_training_set.h5'

exclude_cols = {
    'Uploaded_variation', 'level_1', 'Location', 'Allele', 'Gene',
    'Feature', 'Feature_type', 'Consequence',
    'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids',
    'Codons', 'Existing_variation', 'IMPACT', 'DISTANCE',
    'STRAND', 'FLAGS', 'ClinicalSignificance', 'target', 'MVP_score'
}

model = CNN_Model(
    weights_path=weights_path,
    exclude_cols=exclude_cols,
    train_flag=False,
    verbose=2,
    name='res_HIS_benchmark_training_set',
    fname='../data/test_set_scaled.csv',
    f_out='../data/benchmark_output_data_mode5.csv'
)

model._init_model(verbose=False)
model.model.load_weights(weights_path)
print("Trained model loaded successfully!")






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`
2025-05-10 18:57:48.500493: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-05-10 18:57:48.517919: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3110395000 Hz
2025-05-10 18:57:48.521252: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x61f33f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-05-10 18:57:48.521306: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-05-10 18:57:48.536355: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1





Trained model loaded successfully!


2025-05-10 18:57:49.049049: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-10 18:57:49.049344: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x62a3050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-05-10 18:57:49.049374: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2050, Compute Capability 8.6
2025-05-10 18:57:49.054383: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-10 18:57:49.054423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2050 major: 8 minor: 6 memoryClockRate(GHz): 1.245
pciBusID: 000

In [3]:
# Load data for SHAP analysis
df = pd.read_csv('../data/test_set_scaled.csv')
feature_cols = [col for col in df.columns if col != 'target']
X = df[feature_cols].values

# Sample background and explanation data
np.random.seed(2025)
idx_bg = np.random.choice(X.shape[0], 100, replace=False)
idx_explain = np.random.choice(X.shape[0], 50, replace=False)

X_bg = X[idx_bg].reshape((-1, 42, 1, 1))
X_explain = X[idx_explain].reshape((-1, 42, 1, 1))

# Initialize SHAP DeepExplainer
tf.compat.v1.keras.backend.get_session().run(tf.compat.v1.global_variables_initializer())
explainer = shap.DeepExplainer(model.model, X_bg)
shap_values = explainer.shap_values(X_explain)

# SHAP values for plotting
X_explain_flat = X_explain.reshape((X_explain.shape[0], X_explain.shape[1]))
shap_values_flat = shap_values[0].reshape((shap_values[0].shape[0], shap_values[0].shape[1]))

2025-05-10 18:57:54.655947: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-10 18:57:54.655999: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2050 major: 8 minor: 6 memoryClockRate(GHz): 1.245
pciBusID: 0000:01:00.0
2025-05-10 18:57:54.656110: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2025-05-10 18:57:54.656157: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory
2025-05-10 18:57:54.656187: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could 

In [4]:
# SHAP beeswarm plot
shap.summary_plot(
    shap_values_flat, 
    features=X_explain_flat, 
    feature_names=feature_cols,
    max_display=43,
    show=False
)
plt.savefig('../results/mvp_benchmark_test_shap_beeswarm_summary.png', dpi=300, bbox_inches='tight')
print("SHAP beeswarm plot saved")

SHAP beeswarm plot saved


In [5]:
# SHAP bar chart of mean absolute values
mean_shap = np.abs(shap_values_flat).mean(axis=0)
sorted_idx = np.argsort(mean_shap)
sorted_shap_values = mean_shap[sorted_idx]
sorted_feature_names = np.array(feature_cols)[sorted_idx]

# Define correlated groups
feature_groups = {
    "Group_1": [
        "AlphaMissense_score", "BayesDel_addAF_score", "BayesDel_noAF_score", "CADD_raw", "ClinPred_score",
        "Eigen-PC-raw_coding", "Eigen-raw_coding", "GERP++_RS", "MetaLR_score", "MetaRNN_score",
        "MetaSVM_score", "MutScore_score", "MutationTaster_score", "Polyphen2_HDIV_score",
        "Polyphen2_HVAR_score", "PrimateAI_score", "REVEL_score", "VEST4_score",
        "fathmm-XF_coding_score", "gMVP_score", "phastCons100way_vertebrate",
        "phyloP100way_vertebrate", "phyloP470way_mammalian"
    ],
    "Group_2": ["RegeneronME_ALL_AF", "gnomAD4.1_joint_AF"],
    "Group_3": ["SIFT4G_score", "SIFT_score"]
}

# Define group colors
group_colors = {
    "Group_1": "tomato",
    "Group_2": "mediumseagreen",
    "Group_3": "royalblue",
    "Other": "lightgrey"
}

# Map features to colors
feature_to_color = {}
for group, features in feature_groups.items():
    for feat in features:
        feature_to_color[feat] = group_colors[group]

bar_colors = [feature_to_color.get(f, group_colors["Other"]) for f in sorted_feature_names]

plt.figure(figsize=(8, 18))
plt.barh(sorted_feature_names, sorted_shap_values, color=bar_colors)
plt.xlabel('Mean(|SHAP value|)', fontsize=12)
plt.title('Global Feature Importance (SHAP, Benchmark)', fontsize=14)
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('../results/mvp_benchmark_test_shap_bar_plot_feature_groups.png', dpi=300, bbox_inches='tight')
print("SHAP bar chart saved")

SHAP bar chart saved
